In [0]:
df=spark.sql('SELECT * from college')

In [0]:
df.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: long (nullable = true)
 |-- Accept: long (nullable = true)
 |-- Enroll: long (nullable = true)
 |-- Top10perc: long (nullable = true)
 |-- Top25perc: long (nullable = true)
 |-- F_Undergrad: long (nullable = true)
 |-- P_Undergrad: long (nullable = true)
 |-- Outstate: long (nullable = true)
 |-- Room_Board: long (nullable = true)
 |-- Books: long (nullable = true)
 |-- Personal: long (nullable = true)
 |-- PhD: long (nullable = true)
 |-- Terminal: long (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: long (nullable = true)
 |-- Expend: long (nullable = true)
 |-- Grad_Rate: long (nullable = true)



In [0]:
selected_features=df.select([
 'Private',
 'Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate'])

In [0]:
selected_features=selected_features.na.drop()

In [0]:
selected_features.head()

Out[144]: Row(Private='Yes', Apps=1660, Accept=1232, Enroll=721, Top10perc=23, Top25perc=52, F_Undergrad=2885, P_Undergrad=537, Outstate=7440, Room_Board=3300, Books=450, Personal=2200, PhD=70, Terminal=78, S_F_Ratio=18.1, perc_alumni=12, Expend=7041, Grad_Rate=60)

In [0]:
selected_features.printSchema()

root
 |-- Private: string (nullable = true)
 |-- Apps: long (nullable = true)
 |-- Accept: long (nullable = true)
 |-- Enroll: long (nullable = true)
 |-- Top10perc: long (nullable = true)
 |-- Top25perc: long (nullable = true)
 |-- F_Undergrad: long (nullable = true)
 |-- P_Undergrad: long (nullable = true)
 |-- Outstate: long (nullable = true)
 |-- Room_Board: long (nullable = true)
 |-- Books: long (nullable = true)
 |-- Personal: long (nullable = true)
 |-- PhD: long (nullable = true)
 |-- Terminal: long (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: long (nullable = true)
 |-- Expend: long (nullable = true)
 |-- Grad_Rate: long (nullable = true)



In [0]:
from pyspark.ml.feature import VectorAssembler,StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
private_indexer=StringIndexer(inputCol="Private",outputCol='Private_indexed')
fitted_indexer=private_indexer.fit(selected_features)
data=fitted_indexer.transform(selected_features)
data.columns


Out[146]: ['Private',
 'Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate',
 'Private_indexed']

In [0]:
vec_assemb=VectorAssembler(inputCols=[
 'Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate'],outputCol='features')



In [0]:
 data=vec_assemb.transform(data)


In [0]:
data=data.select('features','Private_indexed')

In [0]:
DTC=DecisionTreeClassifier(labelCol='Private_indexed')

In [0]:
pipeline=Pipeline(stages=[vec_assemb.transform(data),data.select('features','Private_indexed'),DTC])

---------------------------------------------------------------------------
IllegalArgumentException                  Traceback (most recent call last)
<command-2220049664433725> in <cell line: 1>()
----> 1 pipeline=Pipeline(stages=[vec_assemb.transform(data),data.select('features','Private_indexed'),DTC])

/databricks/spark/python/pyspark/ml/base.py in transform(self, dataset, params)
    260                 return self.copy(params)._transform(dataset)
    261             else:
--> 262                 return self._transform(dataset)
    263         else:
    264             raise TypeError("Params must be a param map but got %s." % type(params))

/databricks/spark/python/pyspark/ml/wrapper.py in _transform(self, dataset)
    398 
    399         self._transfer_params_to_java()
--> 400         return DataFrame(self._java_obj.transform(dataset._jdf), dataset.sparkSession)
    401 
    402 

/databricks/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/java_gateway.py in __call__(self, *args)


In [0]:
selected_features

Out[136]: DataFrame[Private: string, Apps: bigint, Accept: bigint, Enroll: bigint, Top10perc: bigint, Top25perc: bigint, F_Undergrad: bigint, P_Undergrad: bigint, Outstate: bigint, Room_Board: bigint, Books: bigint, Personal: bigint, PhD: bigint, Terminal: bigint, S_F_Ratio: double, perc_alumni: bigint, Expend: bigint, Grad_Rate: bigint]

In [0]:
train,test=data.randomSplit([0.7,0.3])

In [0]:
pipeline.fit(train)

Out[138]: PipelineModel_9cd75c7533de

In [0]:
fitted_model=DTC.fit(train)

In [0]:
fitted_model.evaluate(test)

---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
<command-3274686936521777> in <cell line: 1>()
----> 1 fitted_model.evaluate(test)

AttributeError: 'DecisionTreeClassificationModel' object has no attribute 'evaluate'

In [0]:
results=fitted_model.transform(test)

In [0]:
evaluate.show()

+--------------------+---------------+-------------+--------------------+----------+
|            features|Private_indexed|rawPrediction|         probability|prediction|
+--------------------+---------------+-------------+--------------------+----------+
|[152.0,128.0,75.0...|            0.0|  [242.0,0.0]|           [1.0,0.0]|       0.0|
|[193.0,146.0,55.0...|            0.0|   [24.0,2.0]|[0.92307692307692...|       0.0|
|[202.0,184.0,122....|            0.0|  [242.0,0.0]|           [1.0,0.0]|       0.0|
|[233.0,233.0,153....|            1.0|   [24.0,2.0]|[0.92307692307692...|       0.0|
|[245.0,208.0,125....|            0.0|  [242.0,0.0]|           [1.0,0.0]|       0.0|
|[247.0,189.0,100....|            0.0|  [242.0,0.0]|           [1.0,0.0]|       0.0|
|[281.0,266.0,139....|            0.0|  [242.0,0.0]|           [1.0,0.0]|       0.0|
|[283.0,201.0,97.0...|            0.0|  [242.0,0.0]|           [1.0,0.0]|       0.0|
|[292.0,241.0,96.0...|            0.0|  [242.0,0.0]|           [1

In [0]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator,MulticlassClassificationEvaluator


In [0]:
evaluator=MulticlassClassificationEvaluator(labelCol='Private_indexed',metricName='accuracy')

In [0]:
evaluator.evaluate(results)

Out[114]: 0.9416666666666667